# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096996


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:12,  2.61s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:47,  1.84s/it]

Rendering models:  20%|██        | 6/30 [00:06<00:36,  1.54s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:25,  1.13s/it]

...

Rendering models:  80%|████████  | 24/30 [00:08<00:01,  5.92it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  7.02it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.13it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  9.84it/s]

not-logged-in-a0fe519fd650439057ed    0.036694
not-logged-in-3b0b929f4475fd276561    0.003759
not-logged-in-8eae5b2dbfb8c457c47a    0.004023
Alexander00                           0.002493
TateK                                 0.001218
mwalsh868                             0.001328
jnarayanbvg                           0.001191
awright5                              0.002600
thiagoxxxx                            0.002341
aduldulao                             0.003218
not-logged-in-3429675089c67c4fccb8    0.336377
not-logged-in-5f30b2bb9b7f9d49da41    0.007336
clars915                              0.001307
bbradberry08                          0.002263
DarioB                                0.001144
Maverna                               0.001790
pangeli5                              0.000867
ElisabethB                            0.000815
Lavadude                              0.094084
jordanrushworth                       0.001030
not-logged-in-a4bed12d9111c0c34e9e    0.001463
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())